In [1]:
import torch
import math
#torch.set_grad_enabled(False)

In [2]:
#INTERESTING FACT: WHEN COMPUTING THE BATCH GRADIENT PYTORCH DOES NOT TAKE THE SUM OF THE GRADIENTA OF SINGLE DATA POINTS
#BUT THE MEAN. I LEART AT THE EXPENSE OF ABOUT 1H LOL 

In [3]:
class Parameter():
    def __init__(self):
        self.name = ''
        self.data = None
        self.grad = None

In [4]:
class Module(object):
    def forward (self, *input):
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    
    def get_parameters( self ) :
        return []   

In [5]:
class Losses(object):        
    def forward():
        return NotImplementedError
    def backward():
        NotImplementedError

In [6]:
class Optimizer(object):
    def zero_grad(self):
        for parameter in self.param : 
            parameter.grad = 0
            
    def step(self):
        raise NotImplementedError

In [7]:
class SGD(Optimizer):
    # this is a SGD optimizer
    def __init__(self,lr,max_iter, parameters) :  
        super().__init__()
        self.eta = lr
        self.maxStep = max_iter 
        self.param = parameters
        self.number_step = 0

    def step(self): 
        if self.number_step <=self.maxStep:
            for parameter in self.param :
                parameter.data = parameter.data - self.eta * parameter.grad
            self.number_step = self.number_step + 1
        return self.param

In [8]:
class Linear(Module):
    
    def __init__(self, input_dim, out_dim, bias = True):
        super().__init__()
        std = 1/math.sqrt(input_dim)
        self.weight = Parameter()
        self.parameters = []
        
        self.weight.data = torch.rand(out_dim, input_dim)
        self.weight.data = 2*std*self.weight.data - std
        self.weight.name = 'weight'
        self.parameters += [self.weight]
        
        self.with_bias = bias
        if bias :
            self.bias = Parameter()
            self.bias.data = torch.rand(out_dim)
            self.bias.data = 2*std*self.bias.data - std
            self.bias.data = self.bias.data.unsqueeze(0)
            self.bias.name = 'bias'
            self.parameters +=[self.bias]
            
        self.x = None
              
    def forward(self, x):
        self.x = x
        self.batch_size = x.shape[0]
        return self.x.mm(self.weight.data.T) + self.bias.data
        
    def backward(self, prev_grad):
        
        prev_grad = prev_grad.view(self.batch_size, -1, 1)
        print(prev_grad.shape)
        if self.x is None:
            raise CallForwardFirst
        
        if self.weight.grad is None:
            self.weight.grad = torch.zeros_like(self.weight.data)
        
        grad_on_batch = prev_grad.view(self.batch_size, -1, 1)*self.x.view(self.batch_size, 1, -1)
        self.weight.grad += grad_on_batch.mean(0)
        
        if self.with_bias:
            if self.bias.grad is None:
                self.bias.grad = torch.zeros_like(self.bias.data)
            grad_on_batch = prev_grad.view(self.batch_size, -1)
            self.bias.grad += grad_on_batch.mean(0)
        
        #next_grad = prev_grad.view(1, -1)@self.weight.data
        #next_grad = next_grad.view(-1, 1)
        #return next_grad
    
    def get_parameters(self):
        return self.parameters
    

In [9]:
class Tanh(Module):
    def __init__(self):
        self.x = None
    
    def forward (self, x):
        self.x = x
        return torch.tanh(x)
        
    def backward ( self, prev_grad) :
        if self.x is None:
            raise CallForwardFirst
            
        def d(x):
            return 4 * (x.exp() + x.mul(-1).exp()).pow(-2)
        
        return d(self.x)*prev_grad
    

In [10]:
class MSE(Losses):
    # Attention! Works well only when the vectors provided are of the form [batch_size, vector dimension]
    # Otherwise it doesn know what dimesion to pick for the mean computation
    #I'll fix this later
    def __init__(self):
        self.x = None
    def forward(self, x, t):
        self.x = x
        self.t = t
        return (x - t).pow(2).mean()
    
    def backward(self):
        if self.x == None or self.t == None:
            raise CallForwardFirst
        return 2 * (self.x - self.t)/self.x.shape[1]

In [11]:
class Sequential(object):
    def __init__(self, modules):
        super().__init__()
        self.modules=modules
        self.parameters = []
        for m in self.modules:
            param = m.get_parameters()
            if param:
                self.parameters += param
        
    def forward(self,x):
        for m in self.modules:
            x=m.forward(x)
        return x
    
    def backward(self, loss_grad):
        x = loss_grad
        for m in reversed(self.modules):
            x = m.backward(x)
            
    def get_parameters(self):
        return self.parameters

    def set_parameters(self , params):
        self.parameters = params
        

In [12]:
#check that everything is working with loss backward
x = torch.randn(1, 4, requires_grad = False)
y = torch.randn(1, 4,requires_grad = False)

a = x
a.requires_grad_(True)
print(a.requires_grad)

loss_h = MSE()
loss_b = torch.nn.MSELoss()

l_h = loss_h.forward(x, y)
l_b = loss_b(a, y)

l_grad_b = torch.autograd.grad(l_b, a)
l_grad_h = loss_h.backward()
l_grad_b, l_grad_h, 2*(x-y)

True


((tensor([[ 0.9481, -0.0926,  0.0346,  0.5703]]),),
 tensor([[ 0.9481, -0.0926,  0.0346,  0.5703]], grad_fn=<DivBackward0>),
 tensor([[ 3.7925, -0.3704,  0.1382,  2.2814]], grad_fn=<MulBackward0>))

In [13]:
#now check with linear as well

In [14]:
x = torch.randn(5, 9, requires_grad = False)
y = torch.randn(5, 6,requires_grad = False)

linear_h = Linear(9, 6)
linear_b = torch.torch.nn.Linear(9, 6, True)
linear_h.weight.data = linear_b.weight.data
linear_h.bias.data = linear_b.bias.data

output_b = linear_b(x)
l_b = loss_b(output_b, y)
l_b.backward(retain_graph = True)


In [15]:
output_h = linear_h.forward(x)
l_h = loss_h.forward(output_h, y)
linear_h.backward(loss_h.backward())

torch.Size([5, 6, 1])


In [16]:
loss_h.backward()

tensor([[ 0.4649, -0.7647, -0.5509, -0.2238, -0.0072,  0.3812],
        [-0.1325, -0.0872,  0.2394, -0.1855,  0.4789, -0.4686],
        [-0.1473, -0.0965,  0.2050, -0.4820, -0.4123,  0.1531],
        [-0.0361, -0.6278, -0.1105,  0.6435,  0.1467,  0.3227],
        [ 0.2963, -0.1897,  0.0559, -0.1036, -0.0035,  0.0658]])

In [17]:
linear_b.weight.grad, linear_h.weight.grad,  abs(linear_b.weight.grad - linear_h.weight.grad) < 1e-6

(tensor([[ 0.0267,  0.1734,  0.0710,  0.1561,  0.0115,  0.0690,  0.0262, -0.1471,
          -0.0240],
         [ 0.1912,  0.0530, -0.1898, -0.2355,  0.1201, -0.1077,  0.0259,  0.0864,
           0.0656],
         [ 0.1123, -0.0955, -0.0488, -0.2785,  0.0059, -0.1236,  0.0239,  0.1398,
           0.0299],
         [ 0.0369, -0.1038,  0.0170,  0.1754,  0.1569, -0.0439, -0.1270, -0.0260,
          -0.1318],
         [-0.0330, -0.2764,  0.0410,  0.0624, -0.0586, -0.1115, -0.1599, -0.0314,
          -0.0246],
         [-0.0503,  0.2090,  0.0596,  0.1644,  0.0514,  0.1469,  0.0769, -0.0609,
          -0.0533]]),
 tensor([[ 0.0267,  0.1734,  0.0710,  0.1561,  0.0115,  0.0690,  0.0262, -0.1471,
          -0.0240],
         [ 0.1912,  0.0530, -0.1898, -0.2355,  0.1201, -0.1077,  0.0259,  0.0864,
           0.0656],
         [ 0.1123, -0.0955, -0.0488, -0.2785,  0.0059, -0.1236,  0.0239,  0.1398,
           0.0299],
         [ 0.0369, -0.1038,  0.0170,  0.1754,  0.1569, -0.0439, -0.1270, -0.0260

In [18]:
#Sanity check for whats going on when computing the gradient of the weights
l = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
m = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
l.view(5, -1, 1)*m.view(5, 1, -1)

tensor([[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]]])

In [19]:
#check that builtin is doing the sum of the two on pytorch
x = torch.randn(2, 9, requires_grad = False)
y = torch.randn(2, 6,requires_grad = False)

In [20]:
linear_b_1 = torch.torch.nn.Linear(9, 6, True)
linear_b_1.weight.data = linear_b.weight.data
linear_b_1.bias.data = linear_b.bias.data

output_b_1 = linear_b_1(x[0])
l_b_1 = loss_b(output_b_1, y[0])
l_b_1.backward(retain_graph = True)

In [21]:
linear_b_2 = torch.torch.nn.Linear(9, 6, True)
linear_b_2.weight.data = linear_b.weight.data
linear_b_2.bias.data = linear_b.bias.data

output_b_2 = linear_b_2(x[1])
l_b_2 = loss_b(output_b_2, y[1])
l_b_2.backward(retain_graph = True)

In [22]:
linear_b_3 = torch.torch.nn.Linear(9, 6, True)
linear_b_3.weight.data = linear_b.weight.data
linear_b_3.bias.data = linear_b.bias.data

output_b_3 = linear_b_3(x)
l_b_3 = loss_b(output_b_3, y)
l_b_3.backward(retain_graph = True)

In [23]:
linear_b_1.weight.grad, linear_b_2.weight.grad, linear_b_1.weight.grad + linear_b_2.weight.grad

(tensor([[ 0.1492, -0.0264,  0.2193,  0.0035,  0.0215,  0.0057,  0.1597, -0.1664,
          -0.0273],
         [-0.1171,  0.0207, -0.1721, -0.0027, -0.0169, -0.0045, -0.1254,  0.1306,
           0.0214],
         [ 0.5576, -0.0987,  0.8196,  0.0129,  0.0804,  0.0214,  0.5969, -0.6219,
          -0.1021],
         [-0.9310,  0.1648, -1.3685, -0.0216, -0.1343, -0.0358, -0.9967,  1.0385,
           0.1705],
         [ 0.2192, -0.0388,  0.3222,  0.0051,  0.0316,  0.0084,  0.2347, -0.2445,
          -0.0401],
         [ 0.3500, -0.0620,  0.5144,  0.0081,  0.0505,  0.0135,  0.3747, -0.3904,
          -0.0641]]),
 tensor([[-0.0409, -0.0569,  0.0590, -0.0771, -0.0742, -0.0524, -0.0918, -0.0081,
           0.0923],
         [-0.1657, -0.2305,  0.2394, -0.3126, -0.3007, -0.2123, -0.3722, -0.0327,
           0.3743],
         [-0.0480, -0.0668,  0.0693, -0.0905, -0.0871, -0.0615, -0.1078, -0.0095,
           0.1084],
         [ 0.0880,  0.1225, -0.1271,  0.1660,  0.1597,  0.1128,  0.1977,  0.0173

In [24]:
(linear_b_1.bias.grad + linear_b_2.bias.grad)/2 -  linear_b_3.bias.grad

tensor([0.0000e+00, 1.4901e-08, 0.0000e+00, 0.0000e+00, 1.4901e-08, 1.4901e-08])

In [25]:
#check using sequential 
x = torch.randn(5,9, requires_grad = False)
y = torch.randn(5,6,requires_grad = False)

#handmade sequential linear + relu 
linear = Linear(9, 6, True)
sigma = Tanh()
loss = MSE()

net = Sequential([
    linear, 
    sigma
])

output = net.forward(x)
loss.forward(output, y)
net.backward(loss.backward())

torch.Size([5, 6, 1])


In [26]:
a = torch.randn(1, 2)
a, a.mean(0)

(tensor([[0.4537, 0.5416]]), tensor([0.4537, 0.5416]))

In [27]:
b_linear = torch.nn.Linear(9, 6, True)
b_linear.weight.data = linear.weight.data
b_linear.bias.data = linear.bias.data
l = torch.nn.MSELoss()(torch.tanh(b_linear(x)), y)
l.backward()

In [28]:
b_linear.weight.grad, linear.weight.grad,  abs(b_linear.weight.grad - linear.weight.grad).max() 

(tensor([[-0.0343,  0.0278, -0.2554,  0.2128, -0.0976,  0.0224, -0.1785,  0.0978,
           0.0864],
         [ 0.1753,  0.1938, -0.0117, -0.2619,  0.0627, -0.1679,  0.0494, -0.2939,
           0.0582],
         [ 0.2058,  0.0386,  0.1591,  0.1545, -0.0618, -0.0441,  0.1622,  0.0811,
          -0.1307],
         [ 0.2749,  0.1305,  0.1839,  0.0413,  0.1018, -0.1290,  0.1823,  0.1821,
          -0.1829],
         [ 0.0836,  0.0426,  0.1275, -0.2277,  0.1930, -0.0714,  0.1088,  0.0109,
          -0.0579],
         [ 0.2070,  0.0897,  0.1698, -0.1467,  0.1005, -0.1085,  0.1765, -0.0551,
          -0.0836]]),
 tensor([[-0.0343,  0.0278, -0.2554,  0.2128, -0.0976,  0.0224, -0.1785,  0.0978,
           0.0864],
         [ 0.1753,  0.1938, -0.0117, -0.2619,  0.0627, -0.1679,  0.0494, -0.2939,
           0.0582],
         [ 0.2058,  0.0386,  0.1591,  0.1545, -0.0618, -0.0441,  0.1622,  0.0811,
          -0.1307],
         [ 0.2749,  0.1305,  0.1839,  0.0413,  0.1018, -0.1290,  0.1823,  0.1821

In [29]:
b_linear.bias.grad, linear.bias.grad,  abs(b_linear.bias.grad - linear.bias.grad).max()

(tensor([[ 0.0390, -0.0516, -0.1697, -0.0048,  0.0924, -0.0655]]),
 tensor([[ 0.0390, -0.0516, -0.1697, -0.0048,  0.0924, -0.0655]]),
 tensor(1.8626e-08))